## Covid-19 in Europe: Subnational Notification of new cases per 100,000 inhabitants

Data available here:
https://www.ecdc.europa.eu/en/publications-data/subnational-14-day-notification-rate-covid-19

In [1]:
import pandas as pd
import numpy as np

In [11]:
# Focus on first week of January = last year-week in dataset
df = pd.read_csv('Covid19_subnational_14days.csv')
df = df[df['year_week']==df['year_week'].max()]
df

,country,region_name,nuts_code,year_week,rate_14_day_per_100k,source
41,Austria,Burgenland,AT11,2021-W01,280.132091,"Epidemic intelligence, subnational weekly data"
83,Austria,Karnten,AT21,2021-W01,362.428000,"Epidemic intelligence, subnational weekly data"
125,Austria,Niederosterreich,AT12,2021-W01,313.196331,"Epidemic intelligence, subnational weekly data"
167,Austria,Oberosterreich,AT31,2021-W01,295.527615,"Epidemic intelligence, subnational weekly data"
209,Austria,Salzburg,AT32,2021-W01,649.651220,"Epidemic intelligence, subnational weekly data"
...,...,...,...,...,...,...
15052,Sweden,Varmlands Lan,SE311,2021-W01,378.354566,"Epidemic intelligence, subnational weekly data"
15092,Sweden,Vasterbottens Lan,SE331,2021-W01,483.427971,"Epidemic intelligence, subnational weekly data"
15132,Sweden,Vasternorrlands Lan,SE321,2021-W01,678.745014,"Epidemic intelligence, subnational weekly data"
15172,Sweden,Vastmanland,SE125,2021-W01,666.961147,"Epidemic intelligence, subnational weekly data"


The base code used to generate the Folium Worldmap was retrieved from:
https://towardsdatascience.com/using-python-to-create-a-world-map-from-a-list-of-country-names-cd7480d03b10

In [3]:
# Transform country + regional information into geolocation
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="your_app_name")

def geolocate(country):
    try:
        # Geolocate the center of the region
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan, np.nan

df['country_region'] = df['country'] + ', ' + df['region_name']
df['Latitude', 'Longitude'] = df['country_region'].apply(lambda x: geolocate(x))

In [6]:
# Separate Columns for Latitude / Longitude
df['Latitude'] = df.iloc[:,7].apply(lambda x: x[0])
df['Longitude'] = df.iloc[:,7].apply(lambda x: x[1])

import math
df['rate_14_day_per_100k'] = df['rate_14_day_per_100k'].astype('int')

In [8]:
# Create a european subnation map 
import folium
from folium.plugins import MarkerCluster

df_map = df.copy().dropna(how='any')

#empty map
europe_subnational_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(europe_subnational_map)

#create popuptext for each region
for i in range(len(df_map)):
        lat = df_map.iloc[i]['Latitude']
        long = df_map.iloc[i]['Longitude']
        radius=20
        popup_text = """Country : {}<br>
                    Region : {} <br>
                    New cases (per 100,000) in past 14 days: {}<br>"""
        popup_text = popup_text.format(df_map.iloc[i]['country'], df_map.iloc[i]['region_name'],
                                   df_map.iloc[i]['rate_14_day_per_100k'])
        popup_text = folium.Popup(popup_text, max_width=1000, min_width=100)
        folium.CircleMarker(location = [lat, long], radius=radius, popup=popup_text, fill =True).add_to(marker_cluster)
        

#show the map
europe_subnational_map
europe_subnational_map.save('europe_subnational_map.html')
europe_subnational_map.save('europe_subnational_map.png')